# 🧪 LAB-Bench: Evaluate AI on Biology Research Tasks

This notebook evaluates AI models on neurology and biology research capabilities:
- Literature analysis (LitQA)
- Database retrieval (DbQA)
- Figure interpretation (FigQA)
- Table analysis (TableQA)
- Protocol troubleshooting (ProtocolQA)

**Use Case for Neurologists:**
- Benchmark AI assistants for your research needs
- Test models on neurology-specific questions
- Compare different AI models for clinical applications

---

## Step 1: Installation

In [ ]:
# Clone LAB-Bench repository
!git clone https://github.com/Future-House/LAB-Bench.git
%cd LAB-Bench

# Install dependencies
!pip install -q -e .
!pip install -q datasets pillow

print("✅ LAB-Bench installed successfully!")

## Step 2: Setup API Keys

In [ ]:
import os
from google.colab import userdata

# Setup API key for LLM
try:
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API key loaded")
except:
    try:
        os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
        print("✅ Anthropic API key loaded")
    except:
        print("⚠️ No API key found. Add OPENAI_API_KEY or ANTHROPIC_API_KEY to secrets.")

## Step 3: Load LAB-Bench Dataset

In [ ]:
from datasets import load_dataset
import pandas as pd

print("📥 Loading LAB-Bench dataset from Hugging Face...\n")

# Load dataset
dataset = load_dataset("futurehouse/lab-bench", split="train")

print(f"✅ Loaded {len(dataset)} questions")
print(f"\n📊 Dataset Structure:")
print(dataset)

# Convert to pandas for easier exploration
df = pd.DataFrame(dataset)

print(f"\n📋 Categories Available:")
print(df['category'].value_counts())

## Step 4: Explore Sample Questions

In [ ]:
print("🔍 SAMPLE QUESTIONS FROM LAB-BENCH\n")
print("=" * 80)

# Show examples from different categories
categories_to_show = ['LitQA2', 'FigQA', 'ProtocolQA']

for category in categories_to_show:
    category_data = df[df['category'] == category]
    if len(category_data) > 0:
        sample = category_data.iloc[0]
        print(f"\n📌 Category: {category}")
        print("-" * 80)
        print(f"Question: {sample['question']}")
        print(f"\nAnswer Choices:")
        for choice_key in ['choice_A', 'choice_B', 'choice_C', 'choice_D']:
            if choice_key in sample and pd.notna(sample[choice_key]):
                print(f"  {choice_key[-1]}: {sample[choice_key]}")
        print(f"\nCorrect Answer: {sample.get('correct_answer', 'N/A')}")
        print("=" * 80)

## Step 5: Create a Simple AI Agent for Evaluation

In [ ]:
import openai

# Initialize OpenAI client
client = openai.OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

async def simple_agent(question, choices, image=None, **kwargs):
    """
    Simple agent that uses GPT-4o-mini to answer multiple choice questions.
    
    Args:
        question: The question text
        choices: Dict with keys A, B, C, D containing answer choices
        image: Optional PIL image for visual questions
    
    Returns:
        str: Single letter answer (A, B, C, or D)
    """
    
    # Format choices
    choices_text = "\n".join([f"{k}: {v}" for k, v in choices.items() if v])
    
    # Create prompt
    prompt = f"""You are an expert in neurology and biological sciences.

Question: {question}

Answer Choices:
{choices_text}

Please provide ONLY the letter (A, B, C, or D) of the correct answer.
"""
    
    # Call LLM
    messages = [{"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.0,
        max_tokens=10
    )
    
    answer = response.choices[0].message.content.strip().upper()
    
    # Extract just the letter
    for letter in ['A', 'B', 'C', 'D']:
        if letter in answer:
            return letter
    
    return 'A'  # Default fallback

print("✅ Agent created successfully!")

## Step 6: Run Evaluation on Sample Questions

In [ ]:
import asyncio
from tqdm import tqdm

print("🧪 RUNNING EVALUATION\n")
print("=" * 80)

# Evaluate on a small subset (10 questions)
sample_size = 10
sample_data = df.sample(n=min(sample_size, len(df)), random_state=42)

results = []
correct = 0

for idx, row in sample_data.iterrows():
    # Prepare choices
    choices = {
        'A': row.get('choice_A', ''),
        'B': row.get('choice_B', ''),
        'C': row.get('choice_C', ''),
        'D': row.get('choice_D', '')
    }
    
    # Get agent's answer
    predicted = await simple_agent(row['question'], choices)
    
    # Check correctness
    correct_answer = row.get('correct_answer', 'N/A')
    is_correct = (predicted == correct_answer)
    
    if is_correct:
        correct += 1
    
    results.append({
        'Category': row['category'],
        'Question': row['question'][:50] + '...',
        'Predicted': predicted,
        'Correct': correct_answer,
        'Match': '✓' if is_correct else '✗'
    })
    
    print(f"Question {len(results)}/{sample_size}: {row['category']} - {'✓' if is_correct else '✗'}")

# Calculate accuracy
accuracy = (correct / len(results)) * 100

print("\n" + "=" * 80)
print(f"\n📊 EVALUATION RESULTS")
print("=" * 80)
print(f"\nQuestions Evaluated: {len(results)}")
print(f"Correct Answers: {correct}")
print(f"Accuracy: {accuracy:.1f}%")

# Show detailed results
results_df = pd.DataFrame(results)
print("\n\n📋 Detailed Results:\n")
print(results_df.to_string(index=False))

## Step 7: Category-wise Performance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate per-category accuracy
category_performance = results_df.groupby('Category').apply(
    lambda x: (x['Predicted'] == x['Correct']).sum() / len(x) * 100
).reset_index()
category_performance.columns = ['Category', 'Accuracy (%)']

print("📊 CATEGORY-WISE PERFORMANCE\n")
print("=" * 80)
print(category_performance.to_string(index=False))

# Visualization
if len(category_performance) > 1:
    plt.figure(figsize=(10, 6))
    sns.barplot(data=category_performance, x='Category', y='Accuracy (%)')
    plt.title('AI Performance by LAB-Bench Category')
    plt.ylabel('Accuracy (%)')
    plt.xlabel('Category')
    plt.ylim(0, 100)
    plt.axhline(y=25, color='red', linestyle='--', label='Random Chance (25%)')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

## Step 8: Save Results

In [ ]:
from google.colab import drive
import datetime

# Mount Google Drive
drive.mount('/content/drive')

# Create results directory
results_dir = '/content/drive/MyDrive/LAB_Bench_Results'
!mkdir -p "{results_dir}"

# Save results
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
results_df.to_csv(f"{results_dir}/evaluation_results_{timestamp}.csv", index=False)

# Save summary report
report_file = f"{results_dir}/evaluation_summary_{timestamp}.txt"
with open(report_file, 'w') as f:
    f.write("=" * 80 + "\n")
    f.write("LAB-BENCH EVALUATION REPORT\n")
    f.write(f"Generated: {datetime.datetime.now()}\n")
    f.write(f"Model: GPT-4o-mini\n")
    f.write("=" * 80 + "\n\n")
    f.write(f"Overall Accuracy: {accuracy:.1f}%\n")
    f.write(f"Questions Evaluated: {len(results)}\n\n")
    f.write("Category Performance:\n")
    f.write("-" * 80 + "\n")
    f.write(category_performance.to_string(index=False))

print(f"✅ Results saved to Google Drive:")
print(f"   {results_dir}")

## 💡 Next Steps

### For Neurology Research:

**1. Test on More Questions:**
Increase `sample_size` to 50-100 for more robust evaluation

**2. Filter for Neurology-Specific Questions:**
```python
# Filter dataset for neuroscience/neurology topics
neuro_keywords = ['neuron', 'brain', 'cognitive', 'alzheimer', 'parkinson', 'epilepsy']
neuro_questions = df[df['question'].str.lower().str.contains('|'.join(neuro_keywords))]
```

**3. Compare Different Models:**
Test multiple models:
- GPT-4o (more powerful, more expensive)
- Claude-3.5-Sonnet (Anthropic)
- GPT-4o-mini (faster, cheaper)

**4. Add Domain Knowledge:**
Enhance agent with neurology-specific context:
```python
prompt = f"""You are a board-certified neurologist with expertise in 
neurodegenerative diseases, neuroimaging, and clinical neurology.

{question}
"""
```

**5. Integration with Clinical Practice:**
- Benchmark AI assistants before using in clinical research
- Test on your own neurology quiz questions
- Evaluate for specific tasks (e.g., image interpretation, protocol design)